In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def preprocessing():
    glassdoor_df = pd.read_csv('../data/google-amazon-facebook-employee-reviews/employee_reviews.csv', index_col=0)
    glassdoor_df.drop(['link'], axis=1, inplace=True)
    employee_titles_df = glassdoor_df["job-title"].str.split(" - ", n = 1, expand = True)
    employee_titles_df.columns = ['Current-Employee', 'role']
    employee_titles_df['is_current_employee'] = employee_titles_df['Current-Employee'].str.contains("Current Employee").astype(int)
    employee_titles_df.drop(['Current-Employee'], axis=1, inplace=True)
    glassdoor_df = pd.concat([glassdoor_df, employee_titles_df], axis=1)
    glassdoor_df['year'] = glassdoor_df['dates'].str.split(', ').str[1]
    for i, row in glassdoor_df.iterrows():
        if row['year'] == '0000' or pd.isnull(row['year']):
            glassdoor_df.drop([i], inplace=True)
    glassdoor_df['month'] = glassdoor_df['dates'].str.split(', ').str[0].str.split(' ').str[1]
    quarter_list = []
    for _, row in glassdoor_df.iterrows():
        if row['month'] == 'Jan' or row['month'] == 'Feb' or row['month'] == 'Mar':
            quarter_list.append(1)
        elif row['month'] == 'Apr' or row['month'] == 'May' or row['month'] == 'Jun':
            quarter_list.append(2)
        elif row['month'] == 'Jul' or row['month'] == 'Aug' or row['month'] == 'Sep':
            quarter_list.append(3)
        elif row['month'] == 'Oct' or row['month'] == 'Nov' or row['month'] == 'Dec':
            quarter_list.append(4)
        else:
            quarter_list.append(5)
    glassdoor_df['quarter'] = quarter_list
    glassdoor_df.columns = ['company', 'location', 'dates', 'job-title', 'summary', 'pros', 'cons',
                            'advice-to-mgmt', 'overall-ratings', 'work-balance-stars',
                            'culture-values-stars', 'career-opportunities-stars',
                            'comp-benefit-stars', 'senior-management-stars', 'helpful-count', 'role', 'is_current_employee', 'year', 'month', 'quarter']
    for i, row in glassdoor_df.iterrows():
        if row['overall-ratings'] == 'none':
            glassdoor_df.loc[i, 'overall-ratings'] = '-1'
        if row['work-balance-stars'] == 'none':
            glassdoor_df.loc[i, 'work-balance-stars'] = '-1'
        if row['culture-values-stars'] == 'none':
            glassdoor_df.loc[i, 'culture-values-stars'] = '-1'
        if row['career-opportunities-stars'] == 'none':
            glassdoor_df.loc[i, 'career-opportunities-stars'] = '-1'
        if row['comp-benefit-stars'] == 'none':
            glassdoor_df.loc[i, 'comp-benefit-stars'] = '-1'
        if row['senior-management-stars'] == 'none':
            glassdoor_df.loc[i, 'senior-management-stars'] = '-1'
    star_list = ['overall-ratings', 'work-balance-stars', 'culture-values-stars', 'career-opportunities-stars', 'comp-benefit-stars', 'senior-management-stars']
    for feature in star_list:
        glassdoor_df[feature] = glassdoor_df[feature].astype('float')
    for i, row in glassdoor_df.iterrows():
        if row['overall-ratings'] == -1:
            glassdoor_df.loc[i, 'overall-ratings'] = None
        if row['work-balance-stars'] == -1:
            glassdoor_df.loc[i, 'work-balance-stars'] = None
        if row['culture-values-stars'] == -1:
            glassdoor_df.loc[i, 'culture-values-stars'] = None
        if row['career-opportunities-stars'] == -1:
            glassdoor_df.loc[i, 'career-opportunities-stars'] = None
        if row['comp-benefit-stars'] == -1:
            glassdoor_df.loc[i, 'comp-benefit-stars'] = None
        if row['senior-management-stars'] == -1:
            glassdoor_df.loc[i, 'senior-management-stars'] = None
    return glassdoor_df
    
        
    

In [2]:
glassdoor_df = pd.read_csv('../data/google-amazon-facebook-employee-reviews/employee_reviews.csv', index_col=0)

In [3]:
glassdoor_df.drop(['link'], axis=1, inplace=True)

In [4]:
employee_titles_df = glassdoor_df["job-title"].str.split(" - ", n = 1, expand = True)
employee_titles_df.columns = ['Current-Employee', 'role']
employee_titles_df['is_current_employee'] = employee_titles_df['Current-Employee'].str.contains("Current Employee").astype(int)
employee_titles_df.drop(['Current-Employee'], axis=1, inplace=True)

In [6]:
glassdoor_df = pd.concat([glassdoor_df, employee_titles_df], axis=1)

In [7]:
glassdoor_df['year'] = glassdoor_df['dates'].str.split(', ').str[1]

In [8]:
for i, row in glassdoor_df.iterrows():
    if row['year'] == '0000' or pd.isnull(row['year']):
        glassdoor_df.drop([i], inplace=True)

In [9]:
glassdoor_df['month'] = glassdoor_df['dates'].str.split(', ').str[0].str.split(' ').str[1]

In [10]:
quarter_list = []
for _, row in glassdoor_df.iterrows():
    if row['month'] == 'Jan' or row['month'] == 'Feb' or row['month'] == 'Mar':
        quarter_list.append(1)
    elif row['month'] == 'Apr' or row['month'] == 'May' or row['month'] == 'Jun':
        quarter_list.append(2)
    elif row['month'] == 'Jul' or row['month'] == 'Aug' or row['month'] == 'Sep':
        quarter_list.append(3)
    elif row['month'] == 'Oct' or row['month'] == 'Nov' or row['month'] == 'Dec':
        quarter_list.append(4)
    else:
        print(row)
        quarter_list.append(5)

In [11]:
glassdoor_df['quarter'] = quarter_list

In [12]:
glassdoor_df.columns

Index(['company', 'location', 'dates', 'job-title', 'summary', 'pros', 'cons',
       'advice-to-mgmt', 'overall-ratings', 'work-balance-stars',
       'culture-values-stars', 'carrer-opportunities-stars',
       'comp-benefit-stars', 'senior-mangemnet-stars', 'helpful-count', 'role',
       'is_current_employee', 'year', 'month', 'quarter'],
      dtype='object')

In [14]:
glassdoor_df.columns = ['company', 'location', 'dates', 'job-title', 'summary', 'pros', 'cons',
       'advice-to-mgmt', 'overall-ratings', 'work-balance-stars',
       'culture-values-stars', 'career-opportunities-stars',
       'comp-benefit-stars', 'senior-management-stars', 'helpful-count', 'role', 'is_current_employee', 'year', 'month', 'quarter']

In [15]:
for i, row in glassdoor_df.iterrows():
    if row['overall-ratings'] == 'none':
        glassdoor_df.loc[i, 'overall-ratings'] = '-1'
    if row['work-balance-stars'] == 'none':
        glassdoor_df.loc[i, 'work-balance-stars'] = '-1'
    if row['culture-values-stars'] == 'none':
        glassdoor_df.loc[i, 'culture-values-stars'] = '-1'
    if row['career-opportunities-stars'] == 'none':
        glassdoor_df.loc[i, 'career-opportunities-stars'] = '-1'
    if row['comp-benefit-stars'] == 'none':
        glassdoor_df.loc[i, 'comp-benefit-stars'] = '-1'
    if row['senior-management-stars'] == 'none':
        glassdoor_df.loc[i, 'senior-management-stars'] = '-1'
        
    

In [20]:
star_list = ['overall-ratings', 'work-balance-stars', 'culture-values-stars', 'career-opportunities-stars', 'comp-benefit-stars', 'senior-management-stars']
for feature in star_list:
    glassdoor_df[feature] = glassdoor_df[feature].astype('float')

In [46]:
for i, row in glassdoor_df.iterrows():
    if row['overall-ratings'] == -1:
        glassdoor_df.loc[i, 'overall-ratings'] = None
    if row['work-balance-stars'] == -1:
        glassdoor_df.loc[i, 'work-balance-stars'] = None
    if row['culture-values-stars'] == -1:
        glassdoor_df.loc[i, 'culture-values-stars'] = None
    if row['career-opportunities-stars'] == -1:
        glassdoor_df.loc[i, 'career-opportunities-stars'] = None
    if row['comp-benefit-stars'] == -1:
        glassdoor_df.loc[i, 'comp-benefit-stars'] = None
    if row['senior-management-stars'] == -1:
        glassdoor_df.loc[i, 'senior-management-stars'] = None
        
    

### add column for Nones?

### Amazon Reviews + Their Quarterly Earnings

In [3]:
glassdoor_df = preprocessing()

In [4]:
glassdoor_df.to_csv('../data/clean_glassdoor_df.csv')

In [47]:
q_earnings_df = pd.read_csv('../data/amazon_quarterly_earnings.csv')
q_earnings_df['Year'] = q_earnings_df['Year'].astype('str')

In [48]:
amazon_reviews_df = glassdoor_df[glassdoor_df['company'] == 'amazon']

In [49]:
earnings_list = []
for _, row in amazon_reviews_df.iterrows():
    net_income = q_earnings_df['Quarterly Net Income (Billions)'].loc[(q_earnings_df['Quarter'] == row['quarter']) & (q_earnings_df['Year'] == row['year'])]
    if len(net_income.values) > 0:
        earnings_list.append(net_income.values[0])
    else:
        earnings_list.append(None)

In [50]:
amazon_reviews_df['amazon_earnings_this_quarter'] = earnings_list

C:\Users\jq\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
amazon_reviews_df['summary'].isna().any()

True

In [26]:
amazon_reviews_df['advice-to-mgmt'].isna().any()

True

Quarter starts: Jan. 1, Apr. 1, July 1 and Oct. 1.

In [55]:
amazon_reviews_df

,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,...,career-opportunities-stars,comp-benefit-stars,senior-management-stars,helpful-count,role,is_current_employee,year,month,quarter,amazon_earnings_this_quarter
7821,amazon,"Sunnyvale, CA","Dec 10, 2018","Current Employee - Manager, Software Developme...",Software Development Engineering Manager,"Work hard, have fun, make history. Be proud of...",RUS grand each year related to last year's sto...,none,5.0,5.0,...,5.0,4.0,5.0,0,"Manager, Software Development &amp, Engineering",1,2018,Dec,4,3.027
7822,amazon,"Seattle, WA","Jan 10, 2016",Current Employee - Anonymous Employee,You Get What You Put In,"Really smart people, a lot of opportunity for ...",You have to be self motivated. NO ONE will hol...,More on-boarding training before new employees...,5.0,4.0,...,5.0,5.0,5.0,1759,Anonymous Employee,1,2016,Jan,1,0.513
7823,amazon,"Seattle, WA","Feb 20, 2016",Current Employee - Senior Engineering Manager,"Exciting Work, Abusive Culture","Jeff Bezos and his ""S-Team"" are brilliant and ...","The management process is abusive, and I'm cur...",Don't pretend that the recent NY Times article...,3.0,3.0,...,3.0,4.0,4.0,1404,Senior Engineering Manager,1,2016,Feb,1,0.513
7824,amazon,"Seattle, WA","Dec 17, 2013",Current Employee - Software Development Manager,"Can be amazing for some people, horrible for o...",Amazon is doing lot's of cool stuff...but lots...,- You're responsible for your own career progr...,Stack ranking is a horrible practice since it'...,4.0,3.0,...,5.0,5.0,4.0,1275,Software Development Manager,1,2013,Dec,4,0.240
7825,amazon,"Seattle, WA","Mar 23, 2017",Former Employee - Anonymous Employee,We are now in a world where we are condescende...,This company gets A list performance from C li...,Maybe because the company recruits C listers w...,Bezos you bought the Washington Post and hired...,4.0,2.0,...,1.0,5.0,1.0,409,Anonymous Employee,0,2017,Mar,1,0.724
7826,amazon,"Carlisle, PA","May 15, 2017",Former Employee - Tier I Warehouse Associate,"Warehouse associate, full time for over two years",Great starting pay and the ability to get rais...,After 3 years of working with the company you ...,none,4.0,2.0,...,4.0,4.0,5.0,257,Tier I Warehouse Associate,0,2017,May,2,0.197
7827,amazon,"Seattle, WA","Mar 29, 2017",Current Employee - Finance Manager,"Great place to learn, churn n burn culture in ...","- super smart people, the best of the best fro...","- frugality is taken to the extreme, only 2 we...",The culture and reputation of amazon will neve...,4.0,1.0,...,4.0,3.0,3.0,245,Finance Manager,1,2017,Mar,1,0.724
7828,amazon,"Seattle, WA","Dec 7, 2018",Current Employee - Senior Product Manager,Sr. Product manager,Smartest people I’ve ever worked with. The abi...,"Not a con, just hard work. But hard rewarding ...",Reconsider the base salary cap and lack of bas...,5.0,4.0,...,5.0,5.0,4.0,2,Senior Product Manager,1,2018,Dec,4,3.027
7829,amazon,none,"Dec 10, 2018",Current Employee - Anonymous Employee,Central operations,"Weekly pay, great pay,flexible shifts","Fighting for hours, unorganized and bad training",none,4.0,5.0,...,4.0,5.0,5.0,1,Anonymous Employee,1,2018,Dec,4,3.027
7830,amazon,none,"Dec 11, 2018",Current Employee - Warehouse Sortation Associate,Warehouse Sortation Associate,- Company works like a well oiled machine - Ma...,- A lot of the workers don't really care and h...,"Don't just hire everyone, put a bit more effor...",4.0,5.0,...,3.0,4.0,5.0,0,Warehouse Sortation Associate,1,2018,Dec,4,3.027


In [69]:
overall_mean = np.nanmean(amazon_reviews_df['overall-ratings'])
work_balance_mean = np.nanmean(amazon_reviews_df['work-balance-stars'])
culture_mean = np.nanmean(amazon_reviews_df['culture-values-stars'])
career_mean = np.nanmean(amazon_reviews_df['career-opportunities-stars'])
benefit_mean = np.nanmean(amazon_reviews_df['comp-benefit-stars'])
senior_mean = np.nanmean(amazon_reviews_df['senior-management-stars'])

for i, row in amazon_reviews_df.iterrows():
    if row['overall-ratings'] != row['overall-ratings']:
        amazon_reviews_df.loc[i, 'overall-ratings'] = overall_mean
        
    if row['work-balance-stars'] != row['work-balance-stars']:
        amazon_reviews_df.loc[i, 'work-balance-stars'] = work_balance_mean
        
    if row['culture-values-stars'] != row['culture-values-stars']:
        amazon_reviews_df.loc[i, 'culture-values-stars'] = culture_mean
        
    if row['career-opportunities-stars'] != row['career-opportunities-stars']:
        amazon_reviews_df.loc[i, 'career-opportunities-stars'] = career_mean
        
    if row['comp-benefit-stars'] != row['comp-benefit-stars']:
        amazon_reviews_df.loc[i, 'comp-benefit-stars'] = benefit_mean

    if row['senior-management-stars'] != row['senior-management-stars']:
        amazon_reviews_df.loc[i, 'senior-management-stars'] = senior_mean

C:\Users\jq\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [70]:
amazon_reviews_df

,company,location,dates,job-title,summary,pros,cons,advice-to-mgmt,overall-ratings,work-balance-stars,...,career-opportunities-stars,comp-benefit-stars,senior-management-stars,helpful-count,role,is_current_employee,year,month,quarter,amazon_earnings_this_quarter
7821,amazon,"Sunnyvale, CA","Dec 10, 2018","Current Employee - Manager, Software Developme...",Software Development Engineering Manager,"Work hard, have fun, make history. Be proud of...",RUS grand each year related to last year's sto...,none,5.0,5.000000,...,5.000000,4.00000,5.000000,0,"Manager, Software Development &amp, Engineering",1,2018,Dec,4,3.027
7822,amazon,"Seattle, WA","Jan 10, 2016",Current Employee - Anonymous Employee,You Get What You Put In,"Really smart people, a lot of opportunity for ...",You have to be self motivated. NO ONE will hol...,More on-boarding training before new employees...,5.0,4.000000,...,5.000000,5.00000,5.000000,1759,Anonymous Employee,1,2016,Jan,1,0.513
7823,amazon,"Seattle, WA","Feb 20, 2016",Current Employee - Senior Engineering Manager,"Exciting Work, Abusive Culture","Jeff Bezos and his ""S-Team"" are brilliant and ...","The management process is abusive, and I'm cur...",Don't pretend that the recent NY Times article...,3.0,3.000000,...,3.000000,4.00000,4.000000,1404,Senior Engineering Manager,1,2016,Feb,1,0.513
7824,amazon,"Seattle, WA","Dec 17, 2013",Current Employee - Software Development Manager,"Can be amazing for some people, horrible for o...",Amazon is doing lot's of cool stuff...but lots...,- You're responsible for your own career progr...,Stack ranking is a horrible practice since it'...,4.0,3.000000,...,5.000000,5.00000,4.000000,1275,Software Development Manager,1,2013,Dec,4,0.240
7825,amazon,"Seattle, WA","Mar 23, 2017",Former Employee - Anonymous Employee,We are now in a world where we are condescende...,This company gets A list performance from C li...,Maybe because the company recruits C listers w...,Bezos you bought the Washington Post and hired...,4.0,2.000000,...,1.000000,5.00000,1.000000,409,Anonymous Employee,0,2017,Mar,1,0.724
7826,amazon,"Carlisle, PA","May 15, 2017",Former Employee - Tier I Warehouse Associate,"Warehouse associate, full time for over two years",Great starting pay and the ability to get rais...,After 3 years of working with the company you ...,none,4.0,2.000000,...,4.000000,4.00000,5.000000,257,Tier I Warehouse Associate,0,2017,May,2,0.197
7827,amazon,"Seattle, WA","Mar 29, 2017",Current Employee - Finance Manager,"Great place to learn, churn n burn culture in ...","- super smart people, the best of the best fro...","- frugality is taken to the extreme, only 2 we...",The culture and reputation of amazon will neve...,4.0,1.000000,...,4.000000,3.00000,3.000000,245,Finance Manager,1,2017,Mar,1,0.724
7828,amazon,"Seattle, WA","Dec 7, 2018",Current Employee - Senior Product Manager,Sr. Product manager,Smartest people I’ve ever worked with. The abi...,"Not a con, just hard work. But hard rewarding ...",Reconsider the base salary cap and lack of bas...,5.0,4.000000,...,5.000000,5.00000,4.000000,2,Senior Product Manager,1,2018,Dec,4,3.027
7829,amazon,none,"Dec 10, 2018",Current Employee - Anonymous Employee,Central operations,"Weekly pay, great pay,flexible shifts","Fighting for hours, unorganized and bad training",none,4.0,5.000000,...,4.000000,5.00000,5.000000,1,Anonymous Employee,1,2018,Dec,4,3.027
7830,amazon,none,"Dec 11, 2018",Current Employee - Warehouse Sortation Associate,Warehouse Sortation Associate,- Company works like a well oiled machine - Ma...,- A lot of the workers don't really care and h...,"Don't just hire everyone, put a bit more effor...",4.0,5.000000,...,3.000000,4.00000,5.000000,0,Warehouse Sortation Associate,1,2018,Dec,4,3.027


In [71]:
y = amazon_reviews_df['work-balance-stars']

In [72]:
# Not going to use dates, job-title, role, overall-ratings, location 
# for now

X = amazon_reviews_df[['culture-values-stars', 'career-opportunities-stars', 
                       'comp-benefit-stars', 'senior-management-stars', 'helpful-count',
                       'is_current_employee', 'year', 'month', 'quarter', 'amazon_earnings_this_quarter']]